In [13]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [14]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]

In [15]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2
    return Hp
        
Hp = generate_Hp(n, m, d, g)


In [16]:
import sys
sys.path.append('../../')
import z_library.linear_system as ls

In [17]:
def gnrt_gate_hdi(u):
    result = []
    nonzero_indices = ls.find_nonzero_indices(u)
    for urow, nrow in zip(u, nonzero_indices):
        # 把非0元素映射成01
        filtered_arr = [0 if x == -1 else 1 for x in urow if x != 0]
        binary_str = ''.join(map(str, filtered_arr))
        # 取到二进制表示的数
        map_num = int(binary_str, 2)
        # print(map_num)
        length = len(nrow)
        scale = 2**length
        matrix = np.zeros((scale, scale))

        matrix[map_num, scale - 1 - map_num] = 1
        map_num = 2**length - 1 - map_num
        # print(map_num)
        matrix[map_num, scale - 1 - map_num] = 1
        result.append(matrix)
    return result

# print("非零元素的索引：", nonzero_indices)

u = ls.gnrt_u(n, m)
nonzero_indices = ls.find_nonzero_indices(u)
gate_hds = gnrt_gate_hdi(u)
print(nonzero_indices)

[[0, 1, 2, 3, 4, 5], [2, 3, 6, 7], [0, 6, 8], [1, 2, 3, 6, 9]]


In [18]:
gnrt_gate_hdi(u)[1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [19]:
from scipy.linalg import expm
def build_circ(params):
  qc = QuantumCircuit(num_qubits)
  gamma = params
  for i in [1,3,4,6,8,9]:
    qc.x(i)
  for i, (gate_hdi, ind) in enumerate(zip(gate_hds, nonzero_indices)):
    qc.unitary(expm(-1j * gamma[i] * gate_hdi), tuple(num_qubits - 1 - j for j in ind))
  qc.measure_all()
  return qc

In [20]:
ls.set_print_form()
build_circ(np.full(2 * 2, np.pi/3)).draw()

┌──────────┐ ░ ┌─┐         »
    q_0: ─────────────────────────────────────────┤4         ├─░─┤M├─────────»
         ┌───┐                        ┌──────────┐│          │ ░ └╥┘┌─┐      »
    q_1: ┤ X ├────────────────────────┤2         ├┤          ├─░──╫─┤M├──────»
         └───┘            ┌──────────┐│          ││          │ ░  ║ └╥┘┌─┐   »
    q_2: ─────────────────┤3         ├┤          ├┤          ├─░──╫──╫─┤M├───»
         ┌───┐            │          ││          ││          │ ░  ║  ║ └╥┘┌─┐»
    q_3: ┤ X ├────────────┤2         ├┤1         ├┤3         ├─░──╫──╫──╫─┤M├»
         ├───┤┌──────────┐│          ││          ││          │ ░  ║  ║  ║ └╥┘»
    q_4: ┤ X ├┤5         ├┤          ├┤          ├┤  Unitary ├─░──╫──╫──╫──╫─»
         └───┘│          ││  Unitary ││          ││          │ ░  ║  ║  ║  ║ »
    q_5: ─────┤4         ├┤          ├┤  Unitary ├┤          ├─░──╫──╫──╫──╫─»
         ┌───┐│          ││          ││          ││          │ ░  ║  ║  ║  ║ »
    q_6: ┤ X ├┤3         ├┤1         ├┤          ├┤2         ├─░──╫──╫──╫──╫─»
         └───┘│  Unitary ││          ││          ││          │ ░  ║  ║  ║  ║ »
    q_7: ─────┤2         ├┤0         ├┤          ├┤1         ├─░──╫──╫──╫──╫─»
         ┌───┐│          │└──────────┘│          ││          │ ░  ║  ║  ║  ║ »
    q_8: ┤ X ├┤1         ├────────────┤          ├┤0         ├─░──╫──╫──╫──╫─»
         ├───┤│          │            │          │└──────────┘ ░  ║  ║  ║  ║ »
    q_9: ┤ X ├┤0         ├────────────┤0         ├─────────────░──╫──╫──╫──╫─»
         └───┘└──────────┘            └──────────┘             ░  ║  ║  ║  ║ »
meas: 10/═════════════════════════════════════════════════════════╩══╩══╩══╩═»
                                                                  0  1  2  3 »
«                           
«    q_0: ──────────────────
«                           
«    q_1: ──────────────────
«                           
«    q_2: ──────────────────
«                           
«    q_3: ──────────────────
«         ┌─┐               
«    q_4: ┤M├───────────────
«         └╥┘┌─┐            
«    q_5: ─╫─┤M├────────────
«          ║ └╥┘┌─┐         
«    q_6: ─╫──╫─┤M├─────────
«          ║  ║ └╥┘┌─┐      
«    q_7: ─╫──╫──╫─┤M├──────
«          ║  ║  ║ └╥┘┌─┐   
«    q_8: ─╫──╫──╫──╫─┤M├───
«          ║  ║  ║  ║ └╥┘┌─┐
«    q_9: ─╫──╫──╫──╫──╫─┤M├
«          ║  ║  ║  ║  ║ └╥┘
«meas: 10/═╩══╩══╩══╩══╩══╩═
«          4  5  6  7  8  9

In [21]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [22]:
# 初始化迭代计数器
iteration_count = 0
def test(par):
  global iteration_count
  iteration_count = 0
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, par, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.2f}%".format(counts[x] / shots * 100), cost_function(x))

In [23]:
for dep in range(1,10):
  print(f'depth == {dep}')
  test(np.full(dep * len(u), np.pi/3))
  print()

depth == 1


Iteration 10, Result: 5.765
Iteration 20, Result: 5.001
Iteration 30, Result: 5.0
Iteration 40, Result: 5.0
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 5.0
       x: [ 9.908e-01  5.473e-03  1.569e+00  1.011e+00]
    nfev: 44
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0101010000 100.00% 5
1110010110 0.01% 6

depth == 2
Iteration 10, Result: 6.0005
Iteration 20, Result: 5.0005
Iteration 30, Result: 5.022
Iteration 40, Result: 5.001
Iteration 50, Result: 5.0005
Iteration 60, Result: 5.0005
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 5.0005
       x: [ 1.143e+00 -1.375e-02  1.587e+00  1.011e+00  1.260e+00
            1.005e+00  1.005e+00  1.005e+00]
    nfev: 67
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
----------------------